Github: [링크 텍스트](https://github.com/CUN-bjy/lstm-vae-torch)

In [1]:
from google.colab import files
file = files.upload()
!unzip open.zip

Saving open.zip to open.zip
Archive:  open.zip
  inflating: sample_submission.zip   
   creating: test_input/
  inflating: test_input/TEST_01.csv  
  inflating: test_input/TEST_02.csv  
  inflating: test_input/TEST_03.csv  
  inflating: test_input/TEST_04.csv  
  inflating: test_input/TEST_05.csv  
   creating: test_target/
  inflating: test_target/TEST_01.csv  
  inflating: test_target/TEST_02.csv  
  inflating: test_target/TEST_03.csv  
  inflating: test_target/TEST_04.csv  
  inflating: test_target/TEST_05.csv  
   creating: train_input/
  inflating: train_input/CASE_01.csv  
  inflating: train_input/CASE_02.csv  
  inflating: train_input/CASE_03.csv  
  inflating: train_input/CASE_04.csv  
  inflating: train_input/CASE_05.csv  
  inflating: train_input/CASE_06.csv  
  inflating: train_input/CASE_07.csv  
  inflating: train_input/CASE_08.csv  
  inflating: train_input/CASE_09.csv  
  inflating: train_input/CASE_10.csv  
  inflating: train_input/CASE_11.csv  
  inflating: train_input

In [1]:
# Imports
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import glob
from tqdm import tqdm

In [2]:
# LSTM-based Variational Autoencoder
# Input -> Hidden dim -> mean, std -> Parametrization trick -> Decoder -> Output
class LSTMVAE(nn.Module):
  def __init__(self, input_size, hidden_size, latent_size):
    super(LSTMVAE, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.latent_size = latent_size
    self.num_layers = 1

    # encode
    self.lstm_enc = nn.LSTM(input_size=input_size, hidden_size=hidden_size, 
                        num_layers=self.num_layers, batch_first=True)
    self.hid_2mu = nn.Linear(self.hidden_size, self.latent_size)
    self.hid_2sigma = nn.Linear(self.hidden_size, self.latent_size)

    # decode
    self.lstm_dec = nn.LSTM(input_size=latent_size, hidden_size=hidden_size, 
                        num_layers=self.num_layers, batch_first=True)
    self.latent_2hid = nn.Linear(self.latent_size, self.hidden_size)
    self.hid_2out = nn.Linear(hidden_size, input_size)

    self.relu = nn.ReLU()

  def encode(self, x):
    _, hid_cell = self.lstm_enc(x)
    # print(hidden.shape)
    h = hid_cell[0].view(x.size(0), self.hidden_size)  # batch_size x hidden_size
    mu = self.hid_2mu(h)
    sigma = self.hid_2sigma(h)
    return hid_cell, mu, sigma  

  def decode(self, x, hidden_cell):
    output, _ = self.lstm_dec(x, hidden_cell)
    out = self.hid_2out(output)
    return out

  def forward(self, x):
    # Input -> Hidden -> mu, sigma
    hid_cell, mu, sigma = self.encode(x)

    # Reparametrization
    epsilon = torch.randn_like(sigma)
    z_new = mu + sigma*epsilon  # batch_size x latent_size

    # Decode latent space to input space
    z_new = z_new.repeat(1, x.shape[1], 1)
    z_new = z_new.view(x.shape[0], x.shape[1], self.latent_size)
    x_reconstructed = self.decode(z_new, hid_cell)
    return x_reconstructed, mu, sigma

In [20]:
# Configuration
epochs = 50
batch_size = 128
learning_rate = 3e-4
input_size = 15
hidden_size = 256
latent_size = 8

In [4]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

In [5]:
# Load data
all_input_list = sorted(glob.glob('./train_input/CASE_*.csv'))
all_target_list = sorted(glob.glob('./train_target/CASE_*.csv'))

train_input_list = all_input_list[:25]
train_target_list = all_target_list[:25]

val_input_list = all_input_list[25:]
val_target_list = all_target_list[25:]

In [6]:
# Custom dataset

class CustomDataset(Dataset):
  def __init__(self, input_paths, target_paths, infer_mode):  # download, read data
    self.input_paths = input_paths
    self.target_paths = target_paths
    self.infer_mode = infer_mode

    self.data_list = []
    self.label_list = []
    for input_path, target_path in zip(self.input_paths, self.target_paths):
      input_df = pd.read_csv(input_path)
      target_df = pd.read_csv(target_path)

      input_df = input_df.drop(columns=['obs_time'])
      input_df = input_df.fillna(0)

      input_length = int(len(input_df)/24)
      target_length = int(len(target_df))

      for i in range(target_length):
        time_series = input_df[24*i:24*(i+1)].values
        self.data_list.append(torch.Tensor(time_series))

      for label in target_df['predicted_weight_g']:
        self.label_list.append(label)

  def __len__(self):  # return length of data
    return len(self.data_list)

  def __getitem__(self, index):  # return one item on the index
    data = self.data_list[index]
    target = self.label_list[index]
    if self.infer_mode == False:
      return (data, target)
    else:
      return data

In [7]:
train_dataset = CustomDataset(train_input_list, train_target_list, False)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = CustomDataset(val_input_list, val_target_list, False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [21]:
model = LSTMVAE(input_size, hidden_size, latent_size).to(device)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [22]:
# Train network
for epoch in range(epochs):
  loop = tqdm(enumerate(train_loader))
  for i, (x, _) in enumerate(train_loader):
    # foward
    x = x.to(device)
    x_reconstructed, mu, sigma = model(x)

    reconstruction_loss = loss_fn(x_reconstructed, x)
    kl_div = -torch.sum(1 + torch.log(sigma.pow(2)) - mu.pow(2) - sigma.pow(2))
    loss = reconstruction_loss + kl_div

    # backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loop.set_postfix(loss=loss.item())


0it [00:00, ?it/s]
0it [00:00, ?it/s, loss=2.2e+9]
0it [00:38, ?it/s, loss=2.81e+9]

0it [00:00, ?it/s, loss=2.76e+9]
0it [00:00, ?it/s, loss=2.42e+9]
0it [00:00, ?it/s, loss=2.69e+9]
0it [00:00, ?it/s, loss=3.01e+9]
0it [00:00, ?it/s, loss=2.56e+9]
0it [00:00, ?it/s, loss=2.25e+9]
0it [00:00, ?it/s, loss=1.85e+9]
0it [00:00, ?it/s, loss=2.21e+9]
0it [00:00, ?it/s, loss=2.05e+9]
0it [00:00, ?it/s, loss=2.31e+9]
0it [00:00, ?it/s, loss=2.76e+9]
0it [00:00, ?it/s, loss=2.53e+9]
0it [00:00, ?it/s, loss=2.69e+9]
0it [00:00, ?it/s, loss=2.8e+9] 
0it [00:00, ?it/s, loss=2.49e+9]
0it [00:00, ?it/s, loss=2.3e+9] 
0it [00:00, ?it/s, loss=2.14e+9]
0it [00:00, ?it/s, loss=2.61e+9]
0it [00:00, ?it/s, loss=2.69e+9]
0it [00:00, ?it/s, loss=2.29e+9]
0it [00:00, ?it/s, loss=2.32e+9]
0it [00:00, ?it/s]
0it [00:00, ?it/s, loss=2.65e+9]
0it [00:00, ?it/s, loss=3.15e+9]
0it [00:00, ?it/s, loss=2.56e+9]
0it [00:00, ?it/s, loss=2.32e+9]

0it [00:00, ?it/s, loss=2.21e+9]
0it [00:00, ?it/s, loss=2.58e+9]
0it